Напишите функцию get_normalize, которая будет принимать тензор с признаками объектов из какого-то датасета с картинками и будет возвращать поканальное среднее и поканальное стандартное отклонение. Гарантируется, что матрица будет иметь размер [N, C, H, W], где N это количество объектов, C — количество каналов, H, W — размеры изображений. Нужно вернуть кортеж из двух тензоров длины C.

Ваша функция должна иметь следующую сигнатуру def get_normalize(features: torch.Tensor):

In [1]:
import torch

def get_normalize(features: torch.Tensor):
    """
    Вычисляет поканальное среднее и стандартное отклонение для тензора изображений.

    Args:
        features: Тензор размерности [N, C, H, W]

    Returns:
        Кортеж (mean, std) из двух тензоров длины C
    """
    # Вычисляем среднее по измерениям N, H, W (оставляем C)
    mean = torch.mean(features, dim=(0, 2, 3))

    # Вычисляем стандартное отклонение по измерениям N, H, W (оставляем C)
    std = torch.std(features, dim=(0, 2, 3), unbiased=True)

    return mean, std

Напишите функцию get_augmentations, которая будет возвращать готовые аугментации для обучающей выборки и для тестовой выборки. Она должна иметь следующую сигнатуру: def get_augmentations(train: bool = True) -> T.Compose:.

Примените следующие аугментации:

 Измените размер изображения, чтобы его размер был 224 на 224 пикселя (и для обучающей и для тестовой выборки).
 Примените какие-нибудь аугментации из тех, что мы изучили на занятии (только для обучающей).
 Преобразуйте картинку в тензор.
 Примените нормализацию для датасета CIFAR10
Понятно, что изменение размера к 224 на 224 и нормализация для CIFAR10 вместе сочетаются плохо. Поэтому в дальнейшем, когда будете использовать эту функцию для получения аугментаций для вашего нового датасета, не забудьте поменять их на специфичные ему.



In [3]:
pip install torchvision

  Using cached torchvision-0.24.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (5.9 kB)
Using cached torchvision-0.24.0-cp313-cp313-macosx_12_0_arm64.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torchvision.transforms as T

def get_augmentations(train: bool = True) -> T.Compose:
    # Нормализация для CIFAR10 (предварительно вычисленные значения)
    means = (0.49139968, 0.48215841, 0.44653091)
    stds = (0.24703223, 0.24348513, 0.26158784)

    if train:
        # Аугментации для обучающей выборки
        transforms = T.Compose([
            T.Resize((32, 32)),           # Изменение размера до 224x224
            T.RandomHorizontalFlip(p=0.5),  # Случайное горизонтальное отражение
            T.RandomRotation(10),           # Случайный поворот на ±10 градусов
            T.ToTensor(),                   # Преобразование в тензор
            T.Normalize(mean=means, std=stds)  # Нормализация
        ])
    else:
        # Аугментации для тестовой выборки (только базовые преобразования)
        transforms = T.Compose([
            T.Resize((32, 32)),         
            T.ToTensor(),                   # Преобразование в тензор
            T.Normalize(mean=means, std=stds)  # Нормализация
        ])

    return transforms

Напишите функцию predict. Она должна принимать на вход нейронную сеть, даталоадер и torch.device. Она должна иметь следующую сигнатуру: def predict(model: nn.Module, loader: DataLoader, device: torch.device):

Внутри функции сделайте следующие шаги:

Создайте пустой список для хранения предсказаний.
Проитерируйтесь по даталоадеру.
На каждой итерации сделайте forward pass для батча, посчитайте классы как аргмакс по выходу нейросети, по логитам, добавьте тензор с предсказаниями в список.
Сделайте конкатенацию всех предсказаний и верните этот тензор длины N, по числу объектов в датасете.
Ваша функция должна возвращать тензор с классами.

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

def predict(model: nn.Module, loader: DataLoader, device: torch.device) -> torch.Tensor:
    """
    Предсказывает классы для всего датасета.

    Args:
        model: Нейронная сеть
        loader: DataLoader с данными
        device: Устройство для вычислений (cpu/cuda)

    Returns:
        torch.Tensor: Тензор с предсказанными классами длины N
    """
    model.eval()  # Переводим модель в режим оценки
    predictions = []  # Пустой список для хранения предсказаний

    with torch.no_grad():  # Отключаем вычисление градиентов
        for batch in loader:
            # Если даталоадер возвращает (inputs, labels) или только inputs
            if isinstance(batch, (list, tuple)):
                inputs = batch[0]
            else:
                inputs = batch

            # Перемещаем данные на устройство
            inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)

            # Вычисляем argmax по логитам (предсказываем классы)
            _, predicted = torch.max(outputs, 1)

            # Добавляем предсказания в список
            predictions.append(predicted.cpu())  # Перемещаем на CPU для экономии памяти

    # Конкатенируем все предсказания в один тензор
    all_predictions = torch.cat(predictions, dim=0)

    return all_predictions

Обучите модель из двух сверточных слоев на датасете CIFAR10, добейтесь значения метрики Accuracy в 70% на тестовой выборке. Ограничения касаются только количества сверточных слоев в архитектуре модели, можно использовать любые клевые штуки, что мы прошли на занятии. Вам нужно сдать код с функцией, которая возвращает модель, назовите эту функцию create_simple_conv_cifar. Она не принимает аргументов и возвращает модель. Также сдайте предсказание для тестовой выборки датасета CIFAR10, воспользуйтесь функцией predict из предыдущего задания. Воспользуйтесь torch.save для записи тензора с результатом предсказания на диск.

In [29]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

train_dataset = CIFAR10('./data', train=True, transform=get_augmentations(True), download=True)
valid_dataset = CIFAR10('./data', train=False, transform=get_augmentations(False), download=True)


train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def create_simple_conv_cifar() -> nn.Module:
    class SimpleConvModel(nn.Module):
        def __init__(self, num_classes=10):
            super(SimpleConvModel, self).__init__()

            # Первый сверточный блок
            self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
            self.bn1 = nn.BatchNorm2d(64)

            # Второй сверточный блок  
            self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
            self.bn2 = nn.BatchNorm2d(128)

            # Pooling слои
            self.pool = nn.MaxPool2d(2, 2)
            self.dropout = nn.Dropout(0.3)

            # Автоматический расчет размера для полносвязного слоя
            # Для CIFAR10 (32x32): после 2 пулингов: 32->16->8
            self.fc_input_size = 128 * 8 * 8  # 128 каналов * 8 * 8
            self.fc1 = nn.Linear(self.fc_input_size, 256)
            self.fc2 = nn.Linear(256, num_classes)

        def forward(self, x):
            # Первый блок: 32x32 -> 16x16
            x = self.pool(F.relu(self.bn1(self.conv1(x))))
            x = self.dropout(x)

            # Второй блок: 16x16 -> 8x8
            x = self.pool(F.relu(self.bn2(self.conv2(x))))
            x = self.dropout(x)

            # Выпрямление с автоматическим определением размера
            x = x.view(x.size(0), -1)

            # Проверка размера (для отладки)
            if x.size(1) != self.fc_input_size:
                raise ValueError(f"Expected input size {self.fc_input_size}, but got {x.size(1)}. "
                               f"Make sure input images are 32x32 (CIFAR10) and not resized to 224x224.")

            # Полносвязные слои
            x = F.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.fc2(x)

            return x
    
    return SimpleConvModel()

In [31]:
def train_model(model, train_loader, test_loader, device, epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

    train_losses = []
    test_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Валидация
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        train_losses.append(running_loss / len(train_loader))
        test_accuracies.append(accuracy)

        scheduler.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, '
              f'Test Accuracy: {accuracy:.2f}%')

    return train_losses, test_accuracies

In [32]:
model = create_simple_conv_cifar()
print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

Model created with 2,176,010 parameters


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_losses, test_accuracies = train_model(model, train_loader, valid_loader, device, epochs=20)

Epoch 1/20, Loss: 1.6951, Test Accuracy: 51.81%
Epoch 2/20, Loss: 1.3797, Test Accuracy: 59.28%
Epoch 3/20, Loss: 1.2742, Test Accuracy: 63.32%
Epoch 4/20, Loss: 1.2251, Test Accuracy: 64.82%


In [ ]:
predictions = predict(model, valid_loader, device)

torch.save(predictions, 'cifar10_predictions.pt')
print(f"Predictions saved with shape: {predictions.shape}")

Напишите функцию predict_tta. Она должна принимать на вход нейронную сеть, даталоадер, torch.device и количество итераций по даталоадеру. Она должна иметь следующую сигнатуру: def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):.

В этой функции мы применим технику Test Time Augmentation. Основная идея заключается в том, чтобы сделать для каждой картинки из тестовой выборки несколько аугментированных вариантов и сделать для них предсказания. Потом эти предсказания усреднить и использовать как обычно. В синтаксисе PyTorch это вырождается в создание тестового датасета со случайными аугментациями (либо как на обучающей выборке, либо чуть более слабыми). После этого нужно проитерироваться по созданному датасету несколько раз и усреднить ответы модели.

Внутри функции сделайте следующие шаги:

Запустите цикл по количеству итераций.
Внутри цикла проитерируйтесь по даталоадеру.
Запишите ответы (не классы, а сырые выходы нейросети) модели в один большой тензор размера [N, C], где C — число классов, а N — количество объектов в датасете (то есть мы должны иметь для каждого объекта вектор из выходов нейросети, логиты).
Сделайте из этих тензоров один огромный тензор размера [N, C, iterations], усредните его по итерациям, чтобы его размер стал [N, C]
Дальше предскажите классы для объектов по этому тензору как аргмакс, верните их из функции.
Ваша функция должна возвращать тензор с классами. Не забудьте переводить модель в режим применения и навешивать декоратор для выключения подсчета градиентов.

In [10]:
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2) -> torch.Tensor:
    """
    Предсказывает классы с использованием Test Time Augmentation.
    
    Args:
        model: Нейронная сеть
        loader: DataLoader с данными (должен иметь случайные аугментации)
        device: Устройство для вычислений (cpu/cuda)
        iterations: Количество итераций TTA
        
    Returns:
        torch.Tensor: Тензор с предсказанными классами длины N
    """
    model.eval()  # Переводим модель в режим оценки
    
    # Список для хранения всех предсказаний по итерациям
    all_logits = []
    
    with torch.no_grad():  # Отключаем вычисление градиентов
        for iteration in range(iterations):
            print(f"TTA Iteration {iteration + 1}/{iterations}")
            
            iteration_logits = []
            
            for batch in loader:
                # Обрабатываем разные форматы возврата даталоадера
                if isinstance(batch, (list, tuple)):
                    inputs = batch[0]
                else:
                    inputs = batch
                
                inputs = inputs.to(device)
                outputs = model(inputs)  # Получаем логиты [batch_size, num_classes]
                iteration_logits.append(outputs.cpu())
            
            # Конкатенируем все батчи в один тензор [N, C] для текущей итерации
            iteration_tensor = torch.cat(iteration_logits, dim=0)
            all_logits.append(iteration_tensor)
    
    # Собираем все итерации в один тензор [N, C, iterations]
    all_logits_tensor = torch.stack(all_logits, dim=2)  # [N, C, iterations]
    
    # Усредняем по итерациям (dim=2) -> получаем [N, C]
    averaged_logits = torch.mean(all_logits_tensor, dim=2)
    
    # Предсказываем классы как argmax по усредненным логитам
    final_predictions = torch.argmax(averaged_logits, dim=1)
    
    return final_predictions